In [20]:
!pip install skl2onnx onnx onnxruntime openvino-dev boto3 botocore


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pickle

In [3]:
# Path to your pickle file
pickle_file_path = 'data/models/model_education_loan.pkl'

# Load the model
with open(pickle_file_path, 'rb') as file:
    pipeline = pickle.load(file)

/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.2 when using version 1.4.2. This might lead to break

In [4]:
pipeline.named_steps

{'Vectorizer': DictVectorizer(),
 'Standardization': StandardScaler(with_mean=False),
 'Classifier': MLPClassifier(hidden_layer_sizes=[50, 50, 50, 50, 50, 50])}

In [5]:
model = pipeline.named_steps['Classifier']

In [6]:
num_features = len(pipeline.named_steps['Vectorizer'].get_feature_names_out())

In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import numpy as np

# Example input to define the model input type
#example_input = np.array([[0.0, 0.0]])  # Assuming two features for illustration

# Define the initial input type as a FloatTensorType
initial_type = [('input', FloatTensorType([None, num_features]))]

# Convert the scikit-learn pipeline to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("models/education_loan/1/model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [15]:
import onnxruntime as rt
sess = rt.InferenceSession("models/education/1/model.onnx", providers=rt.get_available_providers())

In [38]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or data connection variables are empty.  "
                     "Please check your data connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        print('1')
        for filename in files:
            file_path = os.path.join(root, filename)
            print(file_path)
            relative_path = os.path.relpath(file_path, local_directory)
            print('3')
            print(relative_path)
            s3_key = os.path.join(s3_prefix, relative_path)
            print('4')
            print(s3_key)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            print('5')
            num_files += 1
    return num_files


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [39]:
num_files = upload_directory_to_s3("models", "models")

1
1
1
1
1
models/education/1/model.onnx
3
education/1/model.onnx
4
models/education/1/model.onnx
models/education/1/model.onnx -> models/education/1/model.onnx


S3UploadFailedError: Failed to upload models/education/1/model.onnx to my-storage/models/education/1/model.onnx: An error occurred (503) when calling the PutObject operation (reached max retries: 4): Service Unavailable

In [31]:
local_models_directory = "models"

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3("models", "models")

if num_files == 0:
    raise ValueError("No files uploaded.  Did you finish training and "
                     "saving the model to the \"models\" directory?  "
                     "Check for \"models/education/1/model.onnx\"")

models/education/1/model.onnx -> models/education/1/model.onnx


S3UploadFailedError: Failed to upload models/education/1/model.onnx to my-storage/models/education/1/model.onnx: An error occurred (503) when calling the PutObject operation (reached max retries: 4): Service Unavailable

In [ ]:
list_objects('model')

In [18]:
pip install onnxsim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
!python3 -m onnxsim models/education/1/model.onnx models/education/1/model.onnx

Simplifying...
Traceback (most recent call last):
  File "/usr/lib64/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib64/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/app-root/lib64/python3.9/site-packages/onnxsim/__main__.py", line 5, in <module>
    main()
  File "/opt/app-root/lib64/python3.9/site-packages/onnxsim/onnx_simplifier.py", line 489, in main
    model_opt, check_ok = simplify(
  File "/opt/app-root/lib64/python3.9/site-packages/onnxsim/onnx_simplifier.py", line 199, in simplify
    model_opt_bytes = C.simplify(
onnx.onnx_cpp2py_export.checker.ValidationError: Field 'type' of 'value_info' is required but missing.


In [16]:
import openvino
openvino.convert_model("models/education/1/model.onnx")

GeneralFailure: Check 'false' failed at src/frontends/onnx/frontend/src/frontend.cpp:144:
FrontEnd API failed with GeneralFailure:
OpenVINO does not support the following ONNX operations: ai.onnx.ml.ZipMap, ai.onnx.ml.ArrayFeatureExtractor


In [11]:
converted_model_path ="models/education/1/model.onnx"

# Compile the model for CPU (you can choose manually CPU, GPU etc.)
# or let the engine choose the best available device (AUTO).
compiled_model = core.compile_model(model=converted_model_path)

RuntimeError: Exception from src/inference/src/cpp/core.cpp:121:
Exception from src/inference/src/dev/plugin.cpp:58:
Exception from src/plugins/auto/src/auto_schedule.cpp:441:
[AUTO] compile model failed, CPU:Exception from src/inference/src/dev/plugin.cpp:58:
Check 'false' failed at src/frontends/onnx/frontend/src/frontend.cpp:144:
FrontEnd API failed with GeneralFailure:
OpenVINO does not support the following ONNX operations: ai.onnx.ml.ZipMap, ai.onnx.ml.ArrayFeatureExtractor

; 




In [50]:
!mo --input_model models/education_loan/1/model.onnx --output_dir models/education_loan/

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
[ ERROR ]  -------------------------------------------------
[ ERROR ]  ----------------- INTERNAL ERROR ----------------
[ ERROR ]  Unexpected exception happened.
[ ERROR ]  Please contact Model Optimizer developers and forward the following information:
[ ERROR ]  Check 'false' failed at src/frontends/onnx/frontend/src/frontend.cpp:144:
FrontEnd API failed with GeneralFailure:
OpenVINO does not support the following ONNX operations: ai.onnx.ml.ZipMap, ai.onnx.ml.ArrayF